In [1]:
import nltk
nltk.download('nps_chat')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from sklearn.metrics import accuracy_score
import torch
import pandas as pd
from datetime import datetime
import torchvision
from torch.utils.tensorboard import SummaryWriter
from transformers import RobertaTokenizerFast,RobertaTokenizer, RobertaForSequenceClassification,RobertaForMaskedLM, TextClassificationPipeline,TFRobertaModel
import numpy as np


[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Roberta


In [2]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt')
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.bias', 'clas

In [3]:
from datasets import load_dataset

dataset = load_dataset("dutch_social")


Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
encoded_dataset = dataset.map(lambda e: tokenizer(e['full_text'], truncation=True, padding='max_length'), batched=True)


Loading cached processed dataset at C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f\cache-89caabcda559c3b3.arrow
Loading cached processed dataset at C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f\cache-85948a7f0a229cd7.arrow
Loading cached processed dataset at C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f\cache-6b0dda7a42983ffd.arrow


In [5]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [6]:
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=32)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [9]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['test', 'train', 'validation']"

# word prediction

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('pdelobelle/robbert-v2-dutch-base')
model = RobertaForMaskedLM.from_pretrained('pdelobelle/robbert-v2-dutch-base', return_dict=True)

In [ ]:
sequence = f"Mijn opleiding is erg {tokenizer.mask_token}."


In [ ]:
input = tokenizer.encode(sequence, return_tensors="pt").to( 'cuda' if torch.cuda.is_available() else 'cpu' )
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

In [ ]:
with torch.no_grad():
    token_logits = model(input).logits

In [ ]:
logits = token_logits[0, mask_token_index, :].squeeze()
prob = logits.softmax(dim=0)
values, indeces = prob.topk(k=15, dim=0)

for index, token in enumerate(tokenizer.convert_ids_to_tokens(indeces)):
    print(f"{token:20} | id = {indeces[index]:4} | p = {values[index]}")

# test
https://github.com/iPieter/robbert

In [ ]:
from datasets import load_dataset
import os
dataset = load_dataset("dutch_social")

In [ ]:
# Detect hardware, return appropriate distribution strategy (you can see that it is pretty easy to set up).
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set (always set in Kaggle)
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
MODEL_NAME = "pdelobelle/robbert-v2-dutch-base"
MAX_LEN = 512
ARTIFACTS_PATH = '../artifacts/'

BATCH_SIZE = 8 * strategy.num_replicas_in_sync
EPOCHS = 3

if not os.path.exists(ARTIFACTS_PATH):
    os.makedirs(ARTIFACTS_PATH)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt')
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")

In [ ]:
def roberta_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(text)
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [ ]:
X_train = roberta_encode(dataset['train']['full_text'], tokenizer)

In [ ]:
X_test = roberta_encode(dataset['test']['full_text'], tokenizer)

In [ ]:
y_train = dataset['train']['label']
y_test = dataset['test']['label']

In [ ]:
y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [ ]:
def build_model(n_categories):
    with strategy.scope():
        input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
        input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
        input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

        # Import RoBERTa model from HuggingFace
        roberta_model = TFRobertaModel.from_pretrained(MODEL_NAME)
        x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

        # Huggingface transformers have multiple outputs, embeddings are the first one,
        # so let's slice out the first position
        x = x[0]

        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(256, activation='relu')(x)
        x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)

        model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr=1e-5),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

        return model

In [ ]:
with strategy.scope():
    model = build_model(n_categories)
    model.summary()

In [ ]:
with strategy.scope():
    history = model.fit(X_train,y_train,
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        validation_data=(X_test,y_test))